The algorithm proceeds as follows:

 - first the most informative training examples for each class are selected (prototypes)
 - expand each example into a latent function, given by the prior (this is the step that converts GPs into multi-class GPs)
 - ...
 - calculate the posterior for each class, for each sample.
 
 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os
import re
import pickle
import sklearn
import sys
import string

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier,NearestNeighbors
from sklearn.svm import LinearSVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [ ]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [ ]:
%aimport data.movielens_20m_imdb
%aimport helpers.labels,helpers.neighbours

In [ ]:
from data.movielens_20m_imdb import load_or_get_from_cache
from helpers.labels import truncate_labels
from helpers.neighbours import get_predicted_labels_from_neighbours

In [ ]:
INTERIM_DATA_ROOT = os.path.abspath("../../data/interim/movielens-ml20m-imdb/")
ML_ROOT = "/media/felipe/SAMSUNG/movielens/ml-20m/"
IMDB_ROOT = "/media/felipe/SAMSUNG/imdb/"

PATH_TO_MOVIES = ML_ROOT + "/movies.csv"
PATH_TO_TAG_ASSIGNMENTS = ML_ROOT + "/tags.csv"
PATH_TO_MOVIE_PLOTS = IMDB_ROOT+"/plot.list"

# CONFIGS
MAX_NB_WORDS = 4000
NB_NEIGHBOURS = 3
DISTANCE_METRIC='cosine'
WEIGHTS='distance'
PREPROC=None
STOP_WORDS=None
NORM='l2'

In [ ]:
docs_df = load_or_get_from_cache(PATH_TO_MOVIES,PATH_TO_TAG_ASSIGNMENTS,PATH_TO_MOVIE_PLOTS,INTERIM_DATA_ROOT)

In [ ]:
data = docs_df['plot'].values
labelsets = docs_df["unique_tags"].map(lambda tagstring: tagstring.split(",")).values

In [ ]:
mlb = MultiLabelBinarizer()
mlb.fit(labelsets)

In [ ]:
# I can't put this into a pipeline because NearestNeighbors is not a normal classifier, I think
# I need to customize the pipeline object to be able to call the methods for that class.
vect = CountVectorizer(max_features=MAX_NB_WORDS, preprocessor=PREPROC, stop_words=STOP_WORDS)
tfidf = TfidfTransformer(norm=NORM)
nbrs = NearestNeighbors(n_neighbors=NB_NEIGHBOURS, metric=DISTANCE_METRIC)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,labelsets,test_size=0.25)

In [ ]:
y_train = mlb.transform(y_train)
y_test = mlb.transform(y_test)

In [ ]:
# train
X_train = vect.fit_transform(X_train)
X_train = tfidf.fit_transform(X_train)

In [ ]:
# test
X_test = vect.transform(X_test)
X_test = tfidf.transform(X_test)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_train.shape,y_test.shape